In [102]:
import pandas as pd 
import numpy as np  
from glob import glob 
from tqdm import tqdm 


files = glob('FCU/LBNL_FDD_Dataset_FCU/*')
fault_files = [x.split('FCU/LBNL_FDD_Dataset_FCU/')[-1] for x in files]
fault_file_path = [x for x in files]
fname_df = pd.DataFrame(fault_files, columns=['fname'])
fname_df['fpath'] = fault_file_path

faults = pd.read_csv('features/Faults.csv')
different_faults = list(faults['Fault Type'].unique())
fault_dict = dict(zip(different_faults, range(len(different_faults))))
faults['FaultCode'] = faults['Fault Type'].map(fault_dict)

all_faults = fname_df.merge(faults[[
    'Fault File Name',
    'Fault Type',
    'Fault Intensity ',
    'FaultCode']], 
    left_on='fname',
    right_on='Fault File Name')

# reg_result.sort_values('Testing Accuracy').to_csv('models/results/reg_results.csv', index=False)
# pca_result.sort_values('Testing Accuracy').to_csv('models/results/pca_results.csv', index=False)

# pca_result.Fault = pca_result.Fault.str.replace('FCU_', '')
# reg_result.Fault = reg_result.Fault.str.replace('FCU_', '')

# pca_result.to_csv('models/results/pca_results.csv', index=False)
# reg_result.to_csv('models/results/reg_results.csv', index=False)

pca_result = pd.read_csv('models/results/pca_results.csv')
reg_result = pd.read_csv('models/results/reg_results.csv')

pca_result.Fault = pca_result.Fault.apply(lambda x: 'FCU_'+x)
reg_result.Fault = reg_result.Fault.apply(lambda x: 'FCU_'+x)

In [103]:
af = all_faults[['Fault File Name', 'Fault Type', 'Fault Intensity ', 'FaultCode']].copy()
af['Fault File Name'] = af['Fault File Name'].str.replace('.csv', '', regex=False)
af = af.rename(columns={'Fault File Name': 'Fault'})

pca_res = pca_result.merge(af, on = 'Fault').copy()
reg_res = reg_result.merge(af, on = 'Fault').copy()

In [104]:
vols = ['Testing Accuracy', 'Training Accuracy', 'Testing AUC','Training AUC']
pgf = pca_res.groupby('Fault Type')[vols].mean().round(4).sort_values('Testing Accuracy')
rgf = reg_res.groupby('Fault Type')[vols].mean().round(4).sort_values('Testing Accuracy')

pgf.to_csv('models/results/pca_fault_type_ovr_acc.csv')
rgf.round(4).to_csv('models/results/reg_fault_type_ovr_acc.csv')

rgf

,Testing Accuracy,Training Accuracy,Testing AUC,Training AUC
Fault Type,,,,
Heating Control Reverse Acting,0.6563,0.6563,0.4626,0.4627
Cooling Coil Fouling Air-Side,0.6810,0.6802,0.5014,0.5006
FCU Unstable Control,0.7047,0.7028,0.5352,0.5359
Heating Coil Valve Leaking,0.7340,0.7339,0.5851,0.5842
Cooling Coil Fouling Water-Side,0.7429,0.7430,0.5983,0.5983
Heating Coil Fouling Water-Side,0.7510,0.7513,0.6109,0.6113
Cooling Coil Valve Leaking,0.7773,0.7759,0.6513,0.6500
Cooling Coil Valve Stuck,0.8079,0.8081,0.7001,0.7003
Zone Temperature Sensor Bias,0.8188,0.8188,0.7165,0.7168


In [105]:
reg_res.groupby('Fault Type')[vols].mean().round(4)

,Testing Accuracy,Training Accuracy,Testing AUC,Training AUC
Fault Type,,,,
Cooling Coil Fouling Air-Side,0.6810,0.6802,0.5014,0.5006
Cooling Coil Fouling Water-Side,0.7429,0.7430,0.5983,0.5983
Cooling Coil Valve Leaking,0.7773,0.7759,0.6513,0.6500
Cooling Coil Valve Stuck,0.8079,0.8081,0.7001,0.7003
FCU Unstable Control,0.7047,0.7028,0.5352,0.5359
Fan Outlet Blockage,0.8386,0.8395,0.7491,0.7492
Filter Restriction,0.8815,0.8819,0.8155,0.8155
Heating Coil Fouling Airside,0.8793,0.8791,0.8117,0.8111
Heating Coil Fouling Water-Side,0.7510,0.7513,0.6109,0.6113
